In [ ]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage
import queries

## Building out the Dataframes

In [ ]:
ranked = False
query = queries.get_query_Transfers247(ranked)
df_247 = (fx.connDBAndReturnDF(query)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

In [ ]:
fuzzy = False
query = queries.get_query_UnlinkedNCAA(fuzzy)
df_ncaa = (fx.connDBAndReturnDF(query)).set_index('ID')
df_ncaa.index.name = 'NCAA_ID'
df_ncaa

In [ ]:
indexer = recordlinkage.Index()
indexer = recordlinkage.BlockIndex(on=['KeyPositionGroup'])


In [ ]:
pairs = indexer.index(df_ncaa, df_247)

In [ ]:
from functions import YearOther

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
sumFields.append('StandardizedPosition')
c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
sumFields.append('KeyPositionGroup')
c.add(YearOther('Year', 'Year', label='Year'))
sumFields.append('Year')


In [ ]:
features = c.compute(pairs, df_ncaa, df_247)

In [ ]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

In [ ]:
#features.insert(0, 'sourceID', features.index.get_level_values(0))
#features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['sum'].idxmax()]
    if (data['sum'] > .9):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsNCAATransfer.csv")

## Everything below wrote out Fuzzy Match above a specific threshold (.983)
> Next I have to review the next 500 records for matches.  For unranked, I'm setting the threshold to .98

In [ ]:
links = dfFinal.values.tolist()
links[0]

In [ ]:
for record in links:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 5, 3, record[5], 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence, Transfer)
        VALUES (?,?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()

## Below is to create and save annotations

In [ ]:
fuzzyMI = pd.MultiIndex.from_frame(dfFinal)
recordlinkage.write_annotation_file(
    "../Annotations/Annotations/annotation_ACTransfersSept.json",
    fuzzyMI[0:150],
    df_ac,
    df_247,
    dataset_a_name="All Conference",
    dataset_b_name="Master"
)

In [ ]:
annotation = recordlinkage.read_annotation_file("..//Annotations//Results//result-ncaatransfers.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

In [ ]:
print(annotation_dict[0])

In [ ]:
for record in annotation_dict:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 5, 1, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence, Transfer)
        VALUES (?,?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()